# In-game data extraction

In [432]:
import pandas as pd 
import numpy as np 
import json
import matplotlib.pyplot as plt 
import math

In [195]:
from riotwatcher import LolWatcher, ApiError

In [212]:
regions = {'na': 'na1', 'euw': 'euw1'}
queue_types = [420, 400] #id for ranked and normal draft games respectively

In [384]:
lol_watcher = LolWatcher(API_KEY)

In [367]:
#we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region('euw1')
items_version = versions['n']['item']
champs_version = versions['n']['champion']

#we get the list of all the items in the game for that version
items_list = lol_watcher.data_dragon.items(items_version)
champs_list = lol_watcher.data_dragon.champions(champions_version, full=True)['keys']

In [433]:
def get_player_stats_from_game(gameId, summonerName, region):
    """
    Takes a game ID (int), a summoner name (str) and a region (str) and, through Riot Games' API, retrieves stats from the match from the player in the form of a single line dataframe.
    Stats are: win, kda, kill participation (number of kills by the player vs all the kills from his team), death participation (same logic as for kills), killing sprees (number of times the player has killed multiple champions without dying), gold share (gold earned vs all the gold earned by the team), gold efficiency (gold earned vs value of inventory at the end of the game), role played
    """

    #API request to get the match info
    game = lol_watcher.match.by_id(region, gameId)

    #Initiating variables
    df = pd.DataFrame({'summonerName': [summonerName], 'gameId': [gameId], 'region': [region]})
    n = None
    total_dmg_dealt = 0 
    total_dmg_taken = 0 
    total_gold_earned = 0 
    total_gold_spent = 0
    total_kills = 0
    total_deaths = 0

    for i in range(0,10):
        #Getting the index of the player we are interested in 
        if game['participantIdentities'][i]['player']['summonerName'].lower() == summonerName.lower():
            n = i

    for i in range(0,10):
    #another for loop is necessary because we need to define n first
        
        if game['participants'][i]['stats']['win'] == game['participants'][n]['stats']['win']:
            #Getting team stats of the game
            total_dmg_dealt = total_dmg_dealt + game['participants'][i]['stats']['totalDamageDealtToChampions']
            total_dmg_taken = total_dmg_taken + game['participants'][i]['stats']['totalDamageTaken']
            total_kills = total_kills + game['participants'][i]['stats']['kills']
            total_deaths = total_deaths + game['participants'][i]['stats']['deaths']
            total_gold_earned = total_gold_earned + game['participants'][i]['stats']['goldEarned']
            total_gold_spent = total_gold_spent + game['participants'][i]['stats']['goldSpent']

    #Getting stats of our player
    df['win'] = game['participants'][i]['stats']['win']
    df['kda'] = (game['participants'][n]['stats']['kills'] + game['participants'][i]['stats']['assists']) / game['participants'][i]['stats']['deaths']
    df['kill_participation'] = game['participants'][i]['stats']['kills'] / total_kills
    df['death_participation'] = game['participants'][i]['stats']['deaths'] / total_deaths
    df['killing_sprees'] = game['participants'][i]['stats']['killingSprees']
    df['cs_permin@0-10'] = game['participants'][i]['timeline']['creepsPerMinDeltas']['0-10']
    df['xp_permin@0-10'] = game['participants'][i]['timeline']['xpPerMinDeltas']['0-10']
    df['dmgtaken_permin@0-10'] = game['participants'][i]['timeline']['damageTakenPerMinDeltas']['0-10']
    #Checking if the game is longer than 20min (1200 seconds)
    if game['gameDuration'] > 1200:
        df['cs_permin@10-20'] = game['participants'][i]['timeline']['creepsPerMinDeltas']['10-20']
        df['xp_permin@10-20'] = game['participants'][i]['timeline']['xpPerMinDeltas']['10-20']
        df['dmgtaken_permin@10-20'] = game['participants'][i]['timeline']['damageTakenPerMinDeltas']['10-20']
    else:
        df['cs_permin@10-20'] = math.nan
        df['xp_permin@10-20'] = math.nan
        df['dmgtaken_permin@10-20'] = math.nan   



    #getting the list of items in the player's inventory at the end of the game
    items = [str(game1['participants'][n]['stats']['item0']), str(game1['participants'][n]['stats']['item1']), str(game1['participants'][n]['stats']['item2']), str(game1['participants'][n]['stats']['item3']), str(game1['participants'][n]['stats']['item4']), str(game1['participants'][n]['stats']['item5']), str(game1['participants'][n]['stats']['item6'])]
    items_value = 0
    for item in items:
        if item != '0':
            items_value = items_value + items_list['data'][item]['gold']['total']
    df['gold_efficiency'] = items_value / game['participants'][i]['stats']['goldEarned']
    df['gold_share'] = game['participants'][i]['stats']['goldEarned'] / total_gold_earned

    #Getting the role of the player
    if game['participants'][n]['timeline']['lane'] == 'BOTTOM':
        if game['participants'][n]['timeline']['role'] == 'DUO_CARRY':
            df['role'] = 'BOTTOM_CARRY'
        else: df['role'] = 'BOTTOM_SUPPORT'
    else: df['role'] = game['participants'][n]['timeline']['lane']

    df['champion'] = champs_list[str(game['participants'][n]['championId'])]
        
    return df


In [438]:
def get_player_stats(summonerName, region):
    """
    Takes a summoner name and a region and returns the stats of up to the last 100 games played in the form of dataframe.
    """
    #getting ids of the player
    ids = lol_watcher.summoner.by_name(region, summonerName)

    #getting the last 100 matches played until May 1st 2021
    matches = lol_watcher.match.matchlist_by_account(region = regions['euw'], encrypted_account_id = ids['accountId'], queue = queue_types)

    stats_list = []

    for i in range(0, len(matches['matches'])):
        stats_list.append(get_player_stats_from_game(matches['matches'][i]['gameId'], summonerName, region))
    
    full_stats = pd.concat(stats_list)

    

    full_stats = full_stats.groupby(['summonerName']).agg(

        #Region??
        #Role?? Is role support?
        #Champion?? Is champ Yasuo?

        #percentage of wins
        winrate = ('win', 'mean'),

        #check if the most played role is support (support usually have lower in-game stats because of their role)
        is_support = ('role', lambda role: True if role.value_counts().index[0] == 'BOTTOM_SUPPORT' else False),

        #average and standard deviation to see overall performance and variance in performance
        #KDA
        avg_kda = ('kda', 'mean'),
        std_kda = ('kda', 'std'),

        #Kill participation
        avg_kp = ('kill_participation', 'mean'),
        std_kp = ('kill_participation', 'std'),

        #Death participation
        avg_dp = ('death_participation', 'mean'),
        std_dp = ('death_participation', 'std'),

        #killing sprees
        avg_killing_sprees = ('killing_sprees', 'mean'),

        #CS per min
        avg_cspermin_10 = ('cs_permin@0-10', 'mean'),
        std_cspermin_10 = ('cs_permin@0-10', 'std'),
        avg_cspermin_20 = ('cs_permin@10-20', lambda x: x.dropna().mean()),
        std_cspermin_20 = ('cs_permin@10-20', lambda x: x.dropna().std()),

        #XP per min
        avg_xppermin_10 = ('xp_permin@0-10', 'mean'),
        std_xppermin_10 = ('xp_permin@0-10', 'std'),
        avg_xppermin_20 = ('xp_permin@10-20', lambda x: x.dropna().mean()),
        std_xppermin_20 = ('xp_permin@10-20', lambda x: x.dropna().std()),

        #Damage taken per min
        avg_dmg_takenpermin_10 = ('dmgtaken_permin@0-10', 'mean'),
        std_dmg_takenpermin_10 = ('dmgtaken_permin@0-10', 'std'),
        avg_dmg_takenpermin_20 = ('dmgtaken_permin@10-20', lambda x: x.dropna().mean()),
        std_dmg_takenpermin_20 = ('dmgtaken_permin@10-20', lambda x: x.dropna().std()),

        #Gold share
        avg_gold_share = ('gold_share', 'mean'),
        std_gold_share = ('gold_share', 'std'),

        #Gold efficiency
        avg_gold_efficiency = ('gold_efficiency', 'mean'),
        std_gold_efficiency = ('gold_efficiency', 'std'),
    )

    full_stats['summoner_lvl'] = ids['summonerLevel']

    return full_stats